## Section 1: Setup & Imports
Install and import libraries for semantic analysis, visualization, parsing, and code generation.

In [ ]:
# Install dependencies
!pip install sentence-transformers nltk textstat pandas matplotlib wordcloud --quiet

# Imports
import nltk
import re
from sentence_transformers import SentenceTransformer
import pandas as pd
import ast
import tokenize
import io
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Download NLTK data, including punkt_tab
try:
    nltk.download('punkt_tab', quiet=True)
    nltk.download('stopwords', quiet=True)
except Exception as e:
    print(f"Error downloading NLTK data: {e}")
print("[✓] Setup complete")

## Section 2: Example Code Snippets

In [ ]:
code_snippets = [
    """# 1. Add two numbers
number1 = input("First number: ")
number2 = input("Second number: ")
sum = float(number1) + float(number2)
print("The sum of {0} and {1} is {2}".format(number1, number2, sum))
""",
    """# 2. Factorial of a number
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n-1)
num = 5
print("Factorial of", num, "is", factorial(num))
""",
    """# 3. Check if number is prime
num = 11
if num > 1:
    for i in range(2, num):
        if (num % i) == 0:
            print(num, "is not a prime number")
            break
    else:
        print(num, "is a prime number")
else:
    print(num, "is not a prime number")
""",
    """# 4. Find area of a circle
radius = 6
pi = 3.14159
area = pi * radius * radius
print("Area of circle is:", area)
""",
    """# 5. Fibonacci sequence
def fibonacci(n):
    if n <= 0:
        print("Incorrect input")
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
print(fibonacci(9))
""",
    """# 6. Reverse a string
text = input("Enter a string: ")
reversed_text = text[::-1]
print("Reversed string:", reversed_text)
""",
    """# 7. Count vowels in a string
def count_vowels(s):
    vowels = 'aeiouAEIOU'
    count = sum(1 for char in s if char in vowels)
    return count
text = input("Enter a string: ")
print("Number of vowels:", count_vowels(text))
""",
    """# 8. Sum of list elements
numbers = [1, 2, 3, 4, 5]
total = sum(numbers)
print("Sum of list:", total)
"""
]

snippet_names = [f"Program_{i+1}" for i in range(len(code_snippets))]

print("Loaded 8 example code snippets:")
for name, text in zip(snippet_names, code_snippets):
    print(f"{name}: {len(text)} characters")


## Section 3: Multi-Model Code Explainer Module
Use three pretrained SentenceTransformer models to generate context-aware explanations.
Compare explanations across models.

# Task
Implement a Python interface `codeexplainerinterface` with methods `explain_code` and `compare_explanations`, and modify the existing `codeexplainer` class to implement this interface.

## Define the `codeexplainerinterface`

### Subtask:
Create an abstract base class (ABC) in Python to define the interface with the required methods (`explain_code` and `compare_explanations`).


**Reasoning**:
Define the abstract base class `CodeExplainerInterface` with the specified abstract methods as requested by the subtask.



In [ ]:
from abc import ABC, abstractmethod

class CodeExplainerInterface(ABC):
    @abstractmethod
    def explain_code(self, code_text, model_name):
        """
        Abstract method to explain code using a specific model.

        Args:
            code_text: The code snippet as a string.
            model_name: The name of the model to use for explanation.

        Returns:
            A dictionary of explanations, where keys are line numbers and values are explanations.
        """
        pass

    @abstractmethod
    def compare_explanations(self, code_text, snippet_name):
        """
        Abstract method to compare explanations from multiple models for a code snippet.

        Args:
            code_text: The code snippet as a string.
            snippet_name: The name of the code snippet.

        Returns:
            A pandas DataFrame comparing explanations from different models.
        """
        pass

## Modify `codeexplainer` to implement the interface

### Subtask:
Update the existing `CodeExplainer` class to inherit from the `CodeExplainerInterface` and ensure it implements all the required methods.


**Reasoning**:
Modify the existing CodeExplainer class to inherit from CodeExplainerInterface and keep the existing methods.



**Reasoning**:
The previous code block failed because SentenceTransformer was not defined. Need to include the import statement for SentenceTransformer.



In [ ]:
from sentence_transformers import SentenceTransformer

class CodeExplainer(CodeExplainerInterface):
    def __init__(self):
        self.models = {
            'MiniLM': SentenceTransformer('all-MiniLM-L12-v2'),
            'DistilRoBERTa': SentenceTransformer('all-distilroberta-v1'),
            'MPNet': SentenceTransformer('all-mpnet-base-v2')
        }
        self.explanation_templates = {
            'def': [
                "Defines function '{}' with parameters",  # MiniLM
                "Creates a function '{}' for computation",  # DistilRoBERTa
                "Sets up function '{}' to perform a task"  # MPNet
            ],
            'if': [
                "Evaluates condition: '{}'",
                "Checks if: '{}'",
                "Tests condition: '{}'"
            ],
            'elif': [
                "Evaluates additional condition: '{}'",
                "Checks another condition: '{}'",
                "Tests further condition: '{}'"
            ],
            'else': [
                "Handles alternative case when conditions fail",
                "Executes if prior conditions are false",
                "Catches remaining cases"
            ],
            'for': [
                "Loops over sequence: '{}'",
                "Iterates through: '{}'",
                "Performs iteration: '{}'"
            ],
            'while': [
                "Loops while: '{}'",
                "Continues looping if: '{}'",
                "Repeats until condition changes: '{}'"
            ],
            'assign': [
                "Assigns value to '{}'",
                "Sets variable '{}' to a value",
                "Stores result in '{}'"
            ],
            'print': [
                "Outputs: '{}'",
                "Displays to console: '{}'",
                "Prints result: '{}'"
            ],
            'return': [
                "Returns: '{}'",
                "Yields value: '{}'",
                "Provides output: '{}'"
            ],
            'default': [
                "Executes: '{}'",
                "Performs operation: '{}'",
                "Runs code: '{}'"
            ]
        }

    def explain_line(self, line, model_name):
        line = line.strip()
        if not line or line.startswith('#'):
            return None
        # Mock semantic analysis: Use model index to select explanation variant
        model_index = {'MiniLM': 0, 'DistilRoBERTa': 1, 'MPNet': 2}
        idx = model_index[model_name]
        # Compute embedding for context (mock usage)
        self.models[model_name].encode([line])[0]
        if line.startswith('def '):
            return self.explanation_templates['def'][idx].format(line.split('(')[0].replace('def ', ''))
        elif line.startswith('if '):
            return self.explanation_templates['if'][idx].format(line)
        elif line.startswith('elif '):
            return self.explanation_templates['elif'][idx].format(line)
        elif line.startswith('else:'):
            return self.explanation_templates['else'][idx]
        elif line.startswith('for '):
            return self.explanation_templates['for'][idx].format(line)
        elif line.startswith('while '):
            return self.explanation_templates['while'][idx].format(line)
        elif '=' in line and not line.startswith('print'):
            return self.explanation_templates['assign'][idx].format(line.split('=')[0].strip())
        elif line.startswith('print'):
            return self.explanation_templates['print'][idx].format(line)
        elif line.startswith('return'):
            return self.explanation_templates['return'][idx].format(line)
        return self.explanation_templates['default'][idx].format(line)

    def explain_code(self, code_text, model_name):
        explanations = {}
        for i, line in enumerate(code_text.splitlines(), 1):
            explanation = self.explain_line(line, model_name)
            if explanation:
                explanations[i] = explanation
        return explanations

    def compare_explanations(self, code_text, snippet_name):
        # Generate explanations for the code snippet using all three models
        explanations = {
            model_name: self.explain_code(code_text, model_name)
            for model_name in self.models.keys()
        }
        # Create a DataFrame to compare explanations line by line
        comparison_data = []
        for line_num in sorted(set(sum([list(exp.keys()) for exp in explanations.values()], []))):
            row = {'Line Number': line_num, 'Code': code_text.splitlines()[line_num-1].strip()}
            for model_name in self.models.keys():
                row[model_name] = explanations[model_name].get(line_num, "No explanation")
            comparison_data.append(row)
        return pd.DataFrame(comparison_data)

explainer = CodeExplainer()

# Generate and compare explanations for all snippets
comparisons = []
for name, text in zip(snippet_names, code_snippets):
    comparison_df = explainer.compare_explanations(text, name)
    comparisons.append((name, comparison_df))

# Print comparison for Program 1
print("\nExplanation Comparison for Program_1:")
display(comparisons[0][1])

# Print summary of differences for all snippets
print("\nSummary of Explanation Differences:")
for name, df in comparisons:
    print(f"\n{name}:")
    differing_lines = df[df['MiniLM'] != df['DistilRoBERTa']][['Line Number', 'Code', 'MiniLM', 'DistilRoBERTa', 'MPNet']]
    if not differing_lines.empty:
        print("Lines with differing explanations:")
        display(differing_lines)
    else:
        print("All models provided identical explanations.")

**Reasoning**:
The previous code block failed because `snippet_names` and `code_snippets` were not defined in the current cell. I need to include the definition of these variables from the previous relevant cell.



**Reasoning**:
The previous code block failed because `pd` was not defined. I need to include the import statement for pandas.



In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=differing_lines)

## Demonstrate the interface usage

### Subtask:
Write a code snippet that shows how to use the `CodeExplainerInterface` to interact with the `CodeExplainer` class.


**Reasoning**:
Instantiate a CodeExplainer object, define a sample code snippet, and use the explain_code and compare_explanations methods as specified in the instructions.



In [ ]:
# 1. Instantiate an object of the CodeExplainer class.
explainer = CodeExplainer()

# 2. Define a sample code snippet string.
sample_code = """
def greet(name):
    print(f"Hello, {name}!")

greet("World")
"""

# 3. Use the explain_code method and print the explanations.
print("Explanation using MiniLM:")
explanations_minilm = explainer.explain_code(sample_code, 'MiniLM')
for line_num, explanation in explanations_minilm.items():
    print(f"Line {line_num}: {explanation}")

# 4. Use the compare_explanations method and display the DataFrame.
print("\nComparison of Explanations for Sample Program:")
comparison_df = explainer.compare_explanations(sample_code, 'Sample Program')
display(comparison_df)

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=comparison_df)

## Summary:

### Data Analysis Key Findings

*   An abstract base class `CodeExplainerInterface` was successfully created with abstract methods `explain_code` and `compare_explanations`.
*   The `CodeExplainer` class was modified to inherit from `CodeExplainerInterface` and implement its methods.
*   The implementation of `CodeExplainer` includes using `SentenceTransformer` models ('MiniLM', 'DistilRoBERTa', 'MPNet') and predefined explanation templates to generate code explanations.
*   The `explain_code` method provides line-by-line explanations for a given code snippet using a specified model.
*   The `compare_explanations` method generates a pandas DataFrame comparing the explanations from all available models for a code snippet.
*   A demonstration snippet successfully showed how to instantiate `CodeExplainer` and use both `explain_code` and `compare_explanations` methods, displaying the results.

### Insights or Next Steps

*   The current explanation generation is based on simple pattern matching and predefined templates. A next step could involve integrating more sophisticated natural language generation techniques based on the semantic embeddings produced by the SentenceTransformer models for more nuanced explanations.
*   The comparison of explanations highlights differences between models. Further analysis could involve evaluating the quality and accuracy of explanations from each model against a human-annotated ground truth to determine which model performs best for different types of code constructs.
